In [1]:
import pandas as pd
import math
import numpy as np

import heapq
import numba as nb
from scipy.stats.mstats import gmean, hmean
from state import*

# data_full = pd.read_csv('data_test.csv')
# data_full = data_full.sort_values(by=['TIME', 'PROFIT'], ascending=[False, False], ignore_index=True)
# all_fomula = np.array(pd.read_csv('congthuc.csv')['fomula'])

data_full

c:\Users\Admin\Desktop\SYSTEM\RL_INVEST\state.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  s[1] = 1
c:\Users\Admin\Desktop\SYSTEM\RL_INVEST\state.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  s[2] = 'NOT_INVEST'
c:\Users\Admin\Desktop\SYSTEM\RL_INVEST\state.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  s[4:] = np.average(data_full.loc[id: , data_full.columns[4:]], axis=0)
c:\Users\Admin\Desktop\SYSTEM\RL_INVEST\st

,TIME,PROFIT,SYMBOL,EXCHANGE,MARKET_CAP,Basic earnings per share,Other expenses,Financial expenses,VolumeARG,Financial income,...,Cash and cash equivalents,Short-term trade payables,Taxes and other obligations to the State Budget,Gross profit,Other short-term payables,In which: Loan interest expenses,Sales deductions,Short-term accrued expenses,LIABILITIES,Tangible fixed assets Accumulated depreciation
0,90,1.000000,NOT_INVEST,HOSE,8.873960e+12,231283.594397,8.469288e+09,5.072332e+10,8.857091e+05,6.373902e+10,...,6.778475e+11,5.893811e+11,1.097540e+11,3.105095e+11,4.307930e+11,3.586968e+10,1.243129e+10,3.037054e+11,4.984089e+12,-1.218414e+12
1,90,0.000000,AAA,HOSE,4.893114e+12,0.000000,3.926632e+09,5.918750e+10,5.156710e+06,6.215189e+10,...,1.018940e+12,9.794080e+11,2.943622e+10,3.430300e+11,2.816613e+11,3.589115e+10,3.873387e+09,1.870343e+10,5.338777e+12,-1.579764e+12
2,90,0.000000,AAT,HOSE,8.485598e+11,1384.000000,1.576288e+08,5.314787e+09,1.027330e+06,9.241194e+07,...,3.328005e+10,1.410469e+10,2.253305e+10,1.191474e+11,2.649353e+06,5.314787e+09,0.000000e+00,0.000000e+00,3.428906e+11,-1.813202e+11
3,90,0.000000,ABS,HOSE,9.451310e+11,0.000000,6.931579e+08,1.141712e+10,7.377700e+05,8.641370e+07,...,2.160073e+10,1.738495e+11,2.044294e+10,2.653042e+10,1.491120e+09,0.000000e+00,0.000000e+00,2.410694e+09,7.281878e+11,-7.332759e+09
4,90,0.000000,ACC,HOSE,1.911000e+12,258.000000,9.234700e+04,5.818817e+09,3.242300e+05,2.771447e+10,...,7.737720e+11,8.828534e+10,8.608350e+09,1.338862e+10,8.726839e+10,5.818817e+09,0.000000e+00,1.173808e+10,8.545696e+11,-1.158529e+11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7119,29,0.829231,GIL,HOSE,2.957500e+11,0.000000,1.178573e+07,4.434805e+08,2.435300e+04,1.918037e+08,...,5.592835e+09,2.042547e+10,1.471138e+08,1.327977e+10,9.091083e+08,2.843084e+08,0.000000e+00,2.477158e+08,4.939735e+10,-1.132837e+10
7120,29,0.792308,VSH,HOSE,8.125000e+12,0.000000,0.000000e+00,4.036377e+09,8.040500e+04,1.217429e+10,...,1.117020e+11,7.003186e+09,4.384513e+09,5.236953e+10,3.844896e+08,3.997361e+09,0.000000e+00,3.763020e+09,5.471286e+11,-1.213501e+12
7121,29,0.741068,FPT,HOSE,2.961458e+13,2550.000000,6.714888e+09,1.364417e+10,1.016800e+05,1.010007e+10,...,5.111802e+11,4.444050e+11,1.211761e+11,4.029315e+11,7.813338e+10,9.609989e+09,0.000000e+00,2.484622e+10,1.477624e+12,-2.382139e+11
7122,29,0.724551,GMD,HOSE,7.643971e+12,0.000000,1.093278e+07,9.889580e+09,1.322200e+05,2.829137e+10,...,1.335678e+12,0.000000e+00,0.000000e+00,5.367310e+10,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.744527e+11,-2.710334e+11


In [3]:
data_full.tail(50)

,TIME,PROFIT,SYMBOL,EXCHANGE,MARKET_CAP,Basic earnings per share,Other expenses,Financial expenses,VolumeARG,Financial income,...,Cash and cash equivalents,Short-term trade payables,Taxes and other obligations to the State Budget,Gross profit,Other short-term payables,In which: Loan interest expenses,Sales deductions,Short-term accrued expenses,LIABILITIES,Tangible fixed assets Accumulated depreciation
7074,30,0.994500,NKD,HOSE,2.015956e+12,905.000000,4.799523e+08,2.110358e+09,35978.000000,4.893852e+08,...,1.462323e+10,1.037612e+10,2.955755e+09,2.575967e+10,3.724593e+09,1.930328e+09,6.668523e+08,2.038621e+09,1.519089e+11,-4.500630e+10
7075,30,0.990000,SFC,HOSE,1.700000e+11,875.000000,2.323524e+08,6.016818e+08,38768.000000,3.933979e+07,...,7.867791e+09,1.063386e+10,1.177967e+09,7.509406e+09,1.772677e+09,6.010171e+08,0.000000e+00,1.217681e+08,4.563476e+10,-2.013088e+10
7076,30,0.979710,CII,HOSE,2.070000e+12,812.000000,2.173456e+07,9.998484e+09,43483.000000,1.813713e+10,...,9.205163e+09,7.987285e+08,1.732215e+10,4.382120e+10,3.909705e+10,9.751411e+09,0.000000e+00,0.000000e+00,8.325968e+11,-2.870730e+09
7077,30,0.973239,VIP,HOSE,2.990520e+12,885.000000,0.000000e+00,9.824533e+09,28554.000000,1.392462e+09,...,2.255134e+10,9.218328e+09,2.775436e+08,5.396750e+10,2.452570e+10,9.540308e+09,0.000000e+00,3.400000e+09,7.522876e+11,-3.863862e+11
7078,30,0.961321,ABT,HOSE,6.677999e+11,3526.000000,9.548809e+07,1.027781e+09,35454.000000,4.342655e+09,...,1.913220e+10,9.469906e+09,9.351217e+08,1.875519e+10,5.913651e+09,9.345285e+08,0.000000e+00,0.000000e+00,1.012533e+11,-1.968482e+10
7079,30,0.925974,KDC,HOSE,8.315923e+12,0.000000,8.325895e+08,7.429214e+09,35562.000000,9.947612e+09,...,2.433497e+10,9.411400e+10,4.830990e+09,4.298851e+10,8.356644e+09,6.661165e+09,6.068582e+08,3.236571e+09,4.295843e+11,-1.024408e+11
7080,30,0.815517,VID,HOSE,4.904306e+11,0.000000,1.433379e+07,1.580897e+09,36996.000000,2.581419e+08,...,6.538700e+09,7.207882e+09,2.978521e+09,1.024056e+10,1.608074e+09,1.537355e+09,3.329834e+07,1.569705e+08,1.109069e+11,-3.562153e+10
7081,29,1.711111,BMC,HOSE,7.081560e+11,0.000000,0.000000e+00,0.000000e+00,18710.000000,4.425954e+07,...,3.850054e+10,6.002424e+07,1.062190e+09,1.426318e+10,1.774738e+09,0.000000e+00,1.532980e+09,0.000000e+00,7.134287e+09,-8.930718e+09
7082,29,1.519455,DHG,HOSE,2.056000e+12,3651.000000,2.089633e+08,5.538020e+09,13098.000000,3.636580e+08,...,4.098770e+10,1.879467e+10,2.290466e+08,1.200689e+11,2.559756e+10,5.259272e+09,4.127791e+09,5.027947e+10,3.926887e+11,-3.955948e+10
7083,29,1.493684,DMC,HOSE,1.118150e+12,911.000000,0.000000e+00,2.099700e+09,11120.000000,8.764751e+08,...,1.513942e+10,7.799700e+10,7.650863e+08,3.334393e+10,4.786260e+09,1.316938e+09,2.857209e+08,0.000000e+00,1.663937e+11,-3.162996e+10


In [3]:
result, per_file = normal_main(player_random1, all_fomula,  5,[0])
result, per_file

(array([3., 2.]), [0])

In [6]:
LIST_RANK_CT1, LIST_COMP1 = get_in4_fomula('(-A+T*Y/E)/A')
LIST_RANK_CT2, LIST_COMP2 = get_in4_fomula('(+N*O*X*D)/A/A/A')
data_full['(-A+T*Y/E)/A'] = np.nan_to_num(eval('(-A+T*Y/E)/A'), nan=-math.inf, posinf=-math.inf, neginf=-math.inf)
data_full['(+N*O*X*D)/A/A/A'] = np.nan_to_num(eval('(+N*O*X*D)/A/A/A'), nan=-math.inf, posinf=-math.inf, neginf=-math.inf)


In [1]:
import pandas as pd
import math
import numpy as np

import heapq
import numba as nb
from scipy.stats.mstats import gmean, hmean

In [3]:
a = np.random.rand(4,5)
a 

array([[0.92234437, 0.61133172, 0.70949621, 0.00978018, 0.80054743],
       [0.46089583, 0.70573816, 0.4182623 , 0.06193158, 0.99169704],
       [0.65617142, 0.06335029, 0.62125151, 0.39541802, 0.88598237],
       [0.45747068, 0.30530891, 0.76741461, 0.27089521, 0.26018483]])

In [6]:
b = np.average(a, axis=0)
b 

array([0.62422057, 0.42143227, 0.62910616, 0.18450625, 0.73460292])

In [21]:
data = pd.read_csv('data_test.csv')
data


,TIME,PROFIT,SYMBOL,EXCHANGE,MARKET_CAP,Basic earnings per share,Other expenses,Financial expenses,VolumeARG,Financial income,...,Cash and cash equivalents,Short-term trade payables,Taxes and other obligations to the State Budget,Gross profit,Other short-term payables,In which: Loan interest expenses,Sales deductions,Short-term accrued expenses,LIABILITIES,Tangible fixed assets Accumulated depreciation
0,90,0.000000,AAA,HOSE,4.893114e+12,0.0,3.926632e+09,5.918750e+10,5156710.0,6.215189e+10,...,1.018940e+12,9.794080e+11,2.943622e+10,3.430300e+11,2.816613e+11,3.589115e+10,3.873387e+09,1.870343e+10,5.338777e+12,-1.579764e+12
1,90,0.000000,AAT,HOSE,8.485598e+11,1384.0,1.576288e+08,5.314787e+09,1027330.0,9.241194e+07,...,3.328005e+10,1.410469e+10,2.253305e+10,1.191474e+11,2.649353e+06,5.314787e+09,0.000000e+00,0.000000e+00,3.428906e+11,-1.813202e+11
2,90,0.000000,ABS,HOSE,9.451310e+11,0.0,6.931579e+08,1.141712e+10,737770.0,8.641370e+07,...,2.160073e+10,1.738495e+11,2.044294e+10,2.653042e+10,1.491120e+09,0.000000e+00,0.000000e+00,2.410694e+09,7.281878e+11,-7.332759e+09
3,90,0.000000,ACC,HOSE,1.911000e+12,258.0,9.234700e+04,5.818817e+09,324230.0,2.771447e+10,...,7.737720e+11,8.828534e+10,8.608350e+09,1.338862e+10,8.726839e+10,5.818817e+09,0.000000e+00,1.173808e+10,8.545696e+11,-1.158529e+11
4,90,0.000000,ACL,HOSE,1.040800e+12,673.0,1.132181e+09,1.032095e+10,309440.0,3.054674e+09,...,8.086112e+10,3.258525e+10,1.426303e+10,7.655423e+10,4.100863e+09,1.022072e+10,3.327546e+09,9.132208e+09,7.208971e+11,-4.303955e+11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7057,29,0.970588,TYA,HOSE,2.053272e+11,0.0,1.557920e+06,1.044809e+10,28976.0,1.963448e+09,...,3.196632e+10,6.522285e+10,7.413128e+09,-1.484477e+10,1.660408e+09,1.057408e+10,2.647917e+08,1.181505e+09,7.091034e+11,-1.374839e+11
7058,29,1.043103,VID,HOSE,4.904306e+11,0.0,4.590573e+08,1.721893e+09,23800.0,4.791289e+09,...,3.207240e+09,2.634391e+10,2.451279e+09,8.186806e+09,1.808680e+09,1.672656e+09,1.878944e+07,1.848114e+08,1.002721e+11,-3.356714e+10
7059,29,0.926829,VIP,HOSE,3.453840e+12,0.0,1.008540e+07,1.021192e+10,66803.0,6.500167e+09,...,6.358727e+10,4.932836e+09,5.131970e+08,3.438993e+10,9.711506e+10,9.803599e+09,2.406591e+08,6.335372e+09,8.414399e+11,-3.612094e+11
7060,29,0.951596,VNM,HOSE,3.295183e+13,1968.0,4.666689e+09,1.393602e+10,204121.0,1.623871e+11,...,2.059899e+11,3.053757e+11,3.177571e+10,3.906646e+11,1.835512e+11,1.543970e+09,3.772136e+09,1.468121e+11,8.420871e+11,-8.516176e+11


In [22]:
test = data.loc[7057:, data.columns[4:]]
test

,MARKET_CAP,Basic earnings per share,Other expenses,Financial expenses,VolumeARG,Financial income,General and administration expenses,Short-term prepaid expenses,Selling expenses,Total Other Current Assets,...,Cash and cash equivalents,Short-term trade payables,Taxes and other obligations to the State Budget,Gross profit,Other short-term payables,In which: Loan interest expenses,Sales deductions,Short-term accrued expenses,LIABILITIES,Tangible fixed assets Accumulated depreciation
7057,2.053272e+11,0.0,1.557920e+06,1.044809e+10,28976.0,1.963448e+09,5.185761e+09,4.078282e+09,5.086716e+09,5.444860e+09,...,3.196632e+10,6.522285e+10,7.413128e+09,-1.484477e+10,1.660408e+09,1.057408e+10,2.647917e+08,1.181505e+09,7.091034e+11,-1.374839e+11
7058,4.904306e+11,0.0,4.590573e+08,1.721893e+09,23800.0,4.791289e+09,2.518683e+09,3.643475e+08,6.826015e+08,3.643475e+08,...,3.207240e+09,2.634391e+10,2.451279e+09,8.186806e+09,1.808680e+09,1.672656e+09,1.878944e+07,1.848114e+08,1.002721e+11,-3.356714e+10
7059,3.453840e+12,0.0,1.008540e+07,1.021192e+10,66803.0,6.500167e+09,4.756012e+09,9.654336e+08,2.920165e+09,4.414513e+09,...,6.358727e+10,4.932836e+09,5.131970e+08,3.438993e+10,9.711506e+10,9.803599e+09,2.406591e+08,6.335372e+09,8.414399e+11,-3.612094e+11
7060,3.295183e+13,1968.0,4.666689e+09,1.393602e+10,204121.0,1.623871e+11,5.555869e+10,1.538659e+10,2.104040e+11,5.354723e+10,...,2.059899e+11,3.053757e+11,3.177571e+10,3.906646e+11,1.835512e+11,1.543970e+09,3.772136e+09,1.468121e+11,8.420871e+11,-8.516176e+11
7061,8.125000e+12,0.0,0.000000e+00,4.036377e+09,80405.0,1.217429e+10,1.090348e+09,2.182500e+06,0.000000e+00,3.803682e+09,...,1.117020e+11,7.003186e+09,4.384513e+09,5.236953e+10,3.844896e+08,3.997361e+09,0.000000e+00,3.763020e+09,5.471286e+11,-1.213501e+12


In [19]:
np.average(test, axis=0)

array([ 9.04528475e+12,  3.93600000e+02,  1.02747790e+09,  8.07085793e+09,
        8.08210000e+04,  3.75632677e+10,  1.38218983e+10,  4.15936650e+09,
        4.38186871e+10,  1.35149263e+10,  7.60354322e+10,  7.60655104e+10,
        1.81482104e+10,  2.69010724e+08,  2.51009151e+09,  6.60050475e+10,
        8.32905433e+10,  8.17757069e+10,  9.30756433e+09,  9.41532232e+10,
        5.69039582e+10,  5.51833307e+09,  8.59275238e+08,  3.16553615e+10,
        6.08006222e+11, -5.19475827e+11])

In [20]:
data_full_load = pd.read_csv('data_test.csv')
data_full_load = data_full_load.sort_values(by=['TIME', 'PROFIT'], ascending=[False, False], ignore_index=True)
def get_index_T(data_full):
    list_T = data_full['TIME']
    index_T = [0]
    for i in range(len(list_T)-1):
        if list_T[i] != list_T[i+1]:
            index_T.append(i+1)
    index_T.append(len(list_T))
    return index_T  
index_test = get_index_T(data_full_load)
for id in index_test[:-1]:
    s = data_full_load.iloc[id]
    print(id)


0
202
437
662
911
1102
1316
1502
1664
1821
1970
2107
2240
2385
2523
2645
2768
2885
3011
3169
3311
3464
3600
3712
3832
3953
4091
4194
4324
4421
4516
4616
4729
4822
4913
5010
5102
5165
5230
5303
5351
5407
5524
5574
5633
5672
5739
5844
5933
6074
6242
6339
6490
6586
6657
6677
6724
6774
6811
6881
6969
7020
